In [1]:
pip install transformers

In [2]:
import os

In [3]:
import numpy as np

In [4]:
from PIL import Image,ImageOps

In [5]:
from transformers import ViTFeatureExtractor as FE

In [6]:
img = Image.open(r"D:\download\archive\chest_xray\chest_xray\test\NORMAL\IM-0001-0001.jpeg")

In [7]:
img = img.resize((150,150))

In [8]:
img = ImageOps.colorize(img,white = "white",black = "black")

In [9]:
model = FE.from_pretrained("google/vit-base-patch16-224")

D:\dev\python\Anaconda\Lib\site-packages\transformers\models\vit\feature_extraction_vit.py:28: FutureWarning: The class ViTFeatureExtractor is deprecated and will be removed in version 5 of Transformers. Please use ViTImageProcessor instead.
  warnings.warn(


In [10]:
img = model(images = img)

In [11]:
img = img["pixel_values"]

In [12]:
img = np.array(img)

In [13]:
img = img.flatten()

In [14]:
img.shape

(150528,)

In [15]:
train_dir = r"D:/download/archive/chest_xray/chest_xray/train"
test_dir = r"D:/download/archive/chest_xray/chest_xray/test"
X_train = []
Y_train = []
err_counter = 1

In [16]:
for i in os.listdir(train_dir):
    print(i)
    for j in os.listdir(train_dir + "/" + i):
        try:
            img = Image.open(train_dir + "/" + i + "/" + j)
            img = img.resize((150,150))
            img = ImageOps.colorize(img,white = "white",black = "black")
            img = model(images=img)
            img = img["pixel_values"]
            img = np.array(img)
            X_train.append(img.flatten()/255)
            Y_train.append(i)
        except Exception as e:
            print("the error is ",e,"...counter is ....",err_counter)
            err_counter += 1

NORMAL
PNEUMONIA
the error is   ...counter is .... 1
the error is   ...counter is .... 2
the error is   ...counter is .... 3
the error is   ...counter is .... 4
the error is   ...counter is .... 5
the error is   ...counter is .... 6
the error is   ...counter is .... 7
the error is   ...counter is .... 8
the error is   ...counter is .... 9
the error is   ...counter is .... 10
the error is   ...counter is .... 11
the error is   ...counter is .... 12
the error is   ...counter is .... 13
the error is   ...counter is .... 14
the error is   ...counter is .... 15
the error is   ...counter is .... 16
the error is   ...counter is .... 17
the error is   ...counter is .... 18
the error is   ...counter is .... 19
the error is   ...counter is .... 20
the error is   ...counter is .... 21
the error is   ...counter is .... 22
the error is   ...counter is .... 23
the error is   ...counter is .... 24
the error is   ...counter is .... 25
the error is   ...counter is .... 26
the error is   ...counter is .

the error is   ...counter is .... 220
the error is   ...counter is .... 221
the error is   ...counter is .... 222
the error is   ...counter is .... 223
the error is   ...counter is .... 224
the error is   ...counter is .... 225
the error is   ...counter is .... 226
the error is   ...counter is .... 227
the error is   ...counter is .... 228
the error is   ...counter is .... 229
the error is   ...counter is .... 230
the error is   ...counter is .... 231
the error is   ...counter is .... 232
the error is   ...counter is .... 233
the error is   ...counter is .... 234
the error is   ...counter is .... 235
the error is   ...counter is .... 236
the error is   ...counter is .... 237
the error is   ...counter is .... 238
the error is   ...counter is .... 239
the error is   ...counter is .... 240
the error is   ...counter is .... 241
the error is   ...counter is .... 242
the error is   ...counter is .... 243
the error is   ...counter is .... 244
the error is   ...counter is .... 245
the error is

In [17]:
X_test = []
Y_test = []
err_counter2 = 1
for i in os.listdir(test_dir):
    print(i)
    for j in os.listdir(test_dir + "/" + i):
        try:
            img = Image.open(test_dir + "/" + i + "/" + j)
            img = img.resize((150,150))
            img = ImageOps.colorize(img,white = "white",black = "black")
            img = model(images = img)
            img = img["pixel_values"]
            img = np.array(img)
            X_test.append(img.flatten()/255)
            Y_test.append(i)
        except Exception as e:
            print("the error is ",e,"...counter is ....",err_counter2)
            err_counter2 += 1

NORMAL
PNEUMONIA


In [18]:
from sklearn import linear_model

In [19]:
model = linear_model.LogisticRegression()

In [20]:
model.fit(X_train,Y_train)

LogisticRegression()

In [21]:
pred = model.predict(X_test)

In [22]:
from sklearn.metrics import confusion_matrix

In [23]:
cm = confusion_matrix(Y_test,pred)

In [24]:
cm

array([[ 80, 154],
       [  6, 384]], dtype=int64)

In [25]:
print("accuracy is ",(cm[0,0]+cm[1,1])/(sum(sum(cm))))

accuracy is  0.7435897435897436


In [26]:
from sklearn import ensemble

In [27]:
model = ensemble.RandomForestClassifier()

In [28]:
model.fit(X_train,Y_train)
pred = model.predict(X_test)
cm = confusion_matrix(Y_test,pred)
print(cm)
print("accuracy is ",(cm[0,0]+cm[1,1])/(sum(sum(cm))))

[[ 77 157]
 [  2 388]]
accuracy is  0.7451923076923077


In [1]:
# from sklearn import linear_model, ensemble
from sklearn.metrics import confusion_matrix, roc_curve, auc
from sklearn.preprocessing import LabelEncoder
import matplotlib.pyplot as plt
le = LabelEncoder()
Y_test_binary = le.fit_transform(Y_test)

# 训练第一个模型（Logistic Regression）
model1 = linear_model.LogisticRegression()
model1.fit(X_train, Y_train)
pred_prob1 = model1.predict_proba(X_test)[:, 1]
fpr1, tpr1, thresholds1 = roc_curve(Y_test_binary, pred_prob1)
roc_auc1 = auc(fpr1, tpr1)

# 训练第二个模型（Random Forest）
model2 = ensemble.RandomForestClassifier()
model2.fit(X_train, Y_train)
pred_prob2 = model2.predict_proba(X_test)[:, 1]
fpr2, tpr2, thresholds2 = roc_curve(Y_test_binary, pred_prob2)
roc_auc2 = auc(fpr2, tpr2)

# 绘制ROC曲线
plt.figure(figsize=(8, 8))

# 绘制第一个模型的ROC曲线
plt.plot(fpr1, tpr1, color='darkorange', lw=2, label='Logistic Regression ROC curve (area = {:.2f})'.format(roc_auc1))

# 绘制第二个模型的ROC曲线
plt.plot(fpr2, tpr2, color='green', lw=2, label='Random Forest ROC curve (area = {:.2f})'.format(roc_auc2))

# 绘制对角线
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')

plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic (ROC) Curve')
plt.legend(loc='lower right')
plt.show()

NameError: name 'Y_test' is not defined